In [1]:
import sys
sys.path.insert(0, '..')
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import json
import sys
import pandas as pd
import re
import time

In [2]:
from src.utils import get_full_url, AuditVideo

In [3]:
class Audit(object):
    """
    Static class that includes configuration for our Audit process
    """

    #AUDIT VARIABLES
    USER_AGE = 'young' #'young' or 'old'
    FINANCE_VIDEO_TYPE = 'traditional' #'traditional', 'blockchain', 'mixed', 'unrelated'

    SEED_AGE_VIDEO_PATH = '../data/raw/youtube/seed_age_videos.csv'
    SEED_FINANCE_VIDEO_PATH = '../data/raw/youtube/seed_finance_videos.csv'

    AUDIT_RESULTS_PATH = '../data/audit/youtube/'

In [4]:
#from config import audit

def get_age_seed_videos():
    df = pd.read_csv(Audit.SEED_AGE_VIDEO_PATH, index_col=0)
    df = df[df['label'] == Audit.USER_AGE]
    video_ids = df['video_id'].values
    video_urls = [get_full_url(video_id) for video_id in video_ids]
    return video_urls

def get_finance_seed_videos():
    df = pd.read_csv(Audit.SEED_FINANCE_VIDEO_PATH, index_col=0)
    df = df[df['label'] == Audit.FINANCE_VIDEO_TYPE]
    video_ids = df['video_id'].values
    video_urls = [get_full_url(video_id) for video_id in video_ids]
    return video_urls

get_age_seed_videos()

['https://www.youtube.com/watch?v=W3CDMcWlPyM',
 'https://www.youtube.com/watch?v=-RQdkW-em-g',
 'https://www.youtube.com/watch?v=qN_GMtFhlo4',
 'https://www.youtube.com/watch?v=5TQhslgMt0U',
 'https://www.youtube.com/watch?v=N2EYdmYd4V8',
 'https://www.youtube.com/watch?v=wm6Ld8XcdGc',
 'https://www.youtube.com/watch?v=pWg6aUl0ldg',
 'https://www.youtube.com/watch?v=W8lJMJNOiAI',
 'https://www.youtube.com/watch?v=hSCBHRwFiqM',
 'https://www.youtube.com/watch?v=0lJYviju6RY',
 'https://www.youtube.com/watch?v=gTpogtMdtPU',
 'https://www.youtube.com/watch?v=Muwc8eyxPJs',
 'https://www.youtube.com/watch?v=nmkKo8QAeh4',
 'https://www.youtube.com/watch?v=3ErErK1wRDM',
 'https://www.youtube.com/watch?v=IPUInP53HO0',
 'https://www.youtube.com/watch?v=ViXfbDN6oTI',
 'https://www.youtube.com/watch?v=lUqIPo4DDHE',
 'https://www.youtube.com/watch?v=IAwE4o6gdts',
 'https://www.youtube.com/watch?v=sIxtOqJan2M',
 'https://www.youtube.com/watch?v=3D-vUKVQqS4']

In [5]:
from src.YTDriver import YTDriver

In [6]:
#Record start time
start_time = time.time()
driver = YTDriver(browser='firefox', verbose=True)

#Watch age seed videos
age_seed_videos = get_age_seed_videos()
driver.play_list(age_seed_videos[5:7], homepage_interval=0)

#Watch finance videos
finance_seed_videos = get_finance_seed_videos()
driver.play_list(finance_seed_videos[5:7], homepage_interval=1)

driver.close()

Message: TimedPromise timed out after 30000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:469:5
bail@chrome://remote/content/marionette/sync.sys.mjs:211:19

Ad detected
Skippable ad. Skipping after 3 seconds...
Ad not detected
Ad not detected
Clicking homepage icon
Getting homepage via URL
Ad not detected
Clicking homepage icon
Getting homepage via URL


In [ ]:
def to_csv():
    start_time_str = str(start_time).replace('.', '') #Remove period from start time for filenames

    video_recs_df = pd.DataFrame(driver.video_recs)
    video_recs_df['Start Time'] = start_time
    video_recs_df['Age'] = Audit.USER_AGE
    video_recs_df['Finance Video Type'] = Audit.FINANCE_VIDEO_TYPE
    video_recs_df.to_csv(Audit.AUDIT_RESULTS_PATH + f'{start_time_str}-video_recs.csv', index=False)

    homepage_recs_df = pd.DataFrame(driver.homepage_recs)
    homepage_recs_df['Start Time'] = start_time
    homepage_recs_df['Age'] = Audit.USER_AGE
    homepage_recs_df['Finance Video Type'] = Audit.FINANCE_VIDEO_TYPE
    homepage_recs_df.to_csv(Audit.AUDIT_RESULTS_PATH + f'{start_time_str}-homepage_recs.csv', index=False)

In [ ]:
def scrape_video(url):
    """

    Args:
        url (str): Must be a url for a valid YouTube video
    """
    assert('youtube.com' in url.lower())
    
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    elements = driver.find_elements(By.ID, 'video-title')
    out = []
    for elem in elements[:-1]: #Last element is not actually a video
        title = elem.get_attribute('innerHTML').strip()
        link = elem.find_element(By.XPATH, "./../..").get_attribute('href') #Extracts link to video
        
        row = {'title': title, 'link': link, 'source': url}
        out.append(row)
        
    print(out)
    driver.close()
    return out

def scrape_ytdriver(filepath, recursions=5):
    driver = YTDriver(browser='firefox', verbose=True)

    videos = []
    cur_videos = driver.get_homepage()
    if not cur_videos:
        cur_videos = driver.get_homepage()
    videos += cur_videos
    
    for _ in range(recursions):
        driver.play(cur_videos[0], 10)

        cur_videos = driver.get_recommendations()
        videos += cur_videos
            
    driver.close()

    video_urls = [video.url for video in videos]
    video_urls.insert(0, 'https://www.youtube.com') #Demark start of sessions
    
    list_to_csv(video_urls, filepath)
    return video_urls